This notebook scales the date and generates a sample to test the proto-algo in hls

Imports

In [1]:
import numpy as np
import pandas
import matplotlib.pyplot as plt
import h5py
from multiprocess import Pool
from tqdm import tqdm
import math

import matplotlib
matplotlib.style.use('seaborn')

import sys
sys.path.append('../')
import nonuniform_quantization

File info (modify according to your environment and dataset)

In [2]:
file_path = '/sdf/home/b/berthie/tmox42619_data/h5files/hits.tmox42619.run135.h5'
good_ports = ['port_0','port_1','port_4','port_5','port_12','port_13','port_14','port_15']

Input data

In [3]:
f = h5py.File(file_path, 'r')

raw_tofs = []
for port in good_ports:
    temp = np.asarray(f[port]['tofs'][1:])
    raw_tofs.append(temp-temp.min())      # I subtract the minimum value so it shifts the spectrum "left"

Quantizer (Choose "n", the number of bit for the quantizer)

In [4]:
Q = nonuniform_quantization.quantizer(Nb_bits=5)

Quantizer Training 

In [5]:
with Pool(16) as p:
    pool_outputs = list(
        tqdm(
            p.imap(Q.training,
                   raw_tofs),
            total=len(good_ports)
        )
    ) 

tofs_QLvl = []
for port in range(len(good_ports)):
    tofs_QLvl.append(pool_outputs[port][:])

100%|██████████| 8/8 [00:26<00:00,  3.33s/it]


Quantization

In [6]:
quantized_tofs_cluster_assignments = []
quantized_tofs = []
for port in tqdm(range(len(good_ports))):
    quantized_tofs_temp, quantized_tofs_cluster_assignments_temp = Q.quantization(raw_tofs[port], tofs_QLvl[port], return_cluster_assignments=True)
    quantized_tofs_cluster_assignments.append(quantized_tofs_cluster_assignments_temp)
    quantized_tofs.append(quantized_tofs_temp)

100%|██████████| 8/8 [00:00<00:00, 259.22it/s]


Scaling

This next cell iteratively find the unity resolution we need to represente all possible TOF values over all the dynamic range as a integer value.

In [12]:
assignment_pts_index = []
raw_tofs_index = []
quantized_tofs_index = []
quantized_tofs_index_cluster_assignments = []

for port in tqdm(range(len(good_ports))):
    all_samples =  raw_tofs[port]
    max_value = all_samples.max()

    b=1
    all_samples_normalized = all_samples/max_value
    all_samples_normalized_scaled = np.floor(all_samples_normalized*(2**b)).astype(int)

    while(len(np.unique(all_samples)) != len(np.unique(all_samples_normalized_scaled))):
        b+=1
        all_samples_normalized_scaled = np.floor(all_samples_normalized*(2**b)).astype(int)

    raw_tofs_index_temp = all_samples_normalized_scaled

    assignment_pts = tofs_QLvl[port]
    assignment_pts_normalized = assignment_pts/max_value
    assignment_pts_normalized_scaled = np.floor(assignment_pts_normalized*(2**b)).astype(int)
    assignment_pts_index_temp = assignment_pts_normalized_scaled

    # We can then quantized the same way, but now all values are integer. Note that the cluster assignments are exactly the same as when using the floating values.
    quantized_tofs_index_temp, quantized_tofs_index_cluster_assignments_temp = Q.quantization(raw_tofs_index_temp, assignment_pts_index_temp, return_cluster_assignments = True)

    assignment_pts_index.append(assignment_pts_index_temp)
    raw_tofs_index.append(raw_tofs_index_temp)
    quantized_tofs_index.append(quantized_tofs_index_temp)
    quantized_tofs_index_cluster_assignments.append(quantized_tofs_index_cluster_assignments_temp)


100%|██████████| 8/8 [00:08<00:00,  1.12s/it]


Let's save a sample of data to test the quantizer and the histogram in hls.

The scaling allows for "unsingned long" datatype which aim to lower the ressources usgae on FPGA (note that the quantizer and histogram c code only work as a "stream" at the moment. Metadata, such as "channel, "adresses" and "nedge" are needed to make the histogram) 

In [20]:
def my_hist(x, Q_lvls):
    hist_temp = np.zeros_like(Q_lvls)
    for i in range(len(x)):
        hist_temp[x[i]] += 1
    return hist_temp


cpp_port_test = 0

cpp_q_lvl_ref = assignment_pts_index[cpp_port_test][:].astype(int)
cpp_q_samples_ref = quantized_tofs_index_cluster_assignments[cpp_port_test][0:1000].astype(int)
cpp_raw_samples = quantized_tofs_index[cpp_port_test][0:1000].astype(int)
cpp_hist_ref = my_hist(cpp_q_samples_ref, cpp_q_lvl_ref).astype(int)

np.savetxt('../hls_prototypes/c_files/cpp_hist_ref.dat',cpp_hist_ref,fmt="%u")
np.savetxt('../hls_prototypes/c_files/q_lvls.dat',cpp_q_lvl_ref,fmt="%u")
np.savetxt('../hls_prototypes/c_files/raw.dat',cpp_raw_samples,fmt="%u")
np.savetxt('../hls_prototypes/c_files/q_ref.dat',cpp_q_samples_ref,fmt="%u")


 * MIT License
 *
 * Copyright (c) 2022 SLAC National Accelerator Laboratory
 *
 * Permission is hereby granted, free of charge, to any person obtaining a copy
 * of this software and associated documentation files (the "Software"), to deal
 * in the Software without restriction, including without limitation the rights
 * to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
 * copies of the Software, and to permit persons to whom the Software is
 * furnished to do so, subject to the following conditions:
 *
 * The above copyright notice and this permission notice shall be included in all
 * copies or substantial portions of the Software.
 *
 * THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
 * IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
 * FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
 * AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
 * LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
 * OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
 * SOFTWARE.
 *
 * Authors : Berthié Gouin-Ferland
 * Last update : 2022-12-06
 *
 * Description : This notebook scales the date and generates a sample to test the proto-algo in hls
